#Model Based - Jyoti 202318049

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as ss
from sklearn.model_selection import train_test_split

#DATA PREPROCESSING

In [ ]:
!pip install kaggle
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163017 sha256=41cb27220d02f05a640a557550724c135e342844fb159f4e2dea4bcab7aa21ba
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
movies = pd.read_csv('/content/movie.csv')
ratings = pd.read_csv('/content/rating.csv')

merged_data = pd.merge(ratings, movies, on='movieId')
required_columns=['userId', 'movieId', 'rating']
data= merged_data[required_columns]

In [ ]:
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise.model_selection import cross_validate

# Load the dataset
reader = Reader(line_format='user item rating', sep=',')
data = Dataset.load_from_df(data[['userId', 'movieId', 'rating']], reader)

# Define the SVD algorithm
algo = SVD()
trainset = data.build_full_trainset()  # Build the trainset from the entire dataset
algo.fit(trainset)  # Fit the algorithm to the trainset

# Get the matrices U, S, and V
U = algo.pu
S = np.diag(algo.qi)  # Singular values are stored in qi attribute
V = algo.qi.T

# Compute the reduced matrix
reduced_matrix = np.dot(U, np.dot(S, V))

# 'reduced_matrix' now contains the reduced matrix obtained from SVD

# Perform cross-validation
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9081  0.9184  0.9285  0.9143  0.9283  0.9195  0.0079  
MAE (testset)     0.7010  0.7092  0.7132  0.7044  0.7141  0.7084  0.0051  
Fit time          2.05    1.79    2.01    1.86    4.33    2.41    0.97    
Test time         0.14    0.21    0.13    0.13    0.70    0.26    0.22    


{'test_rmse': array([0.90813726, 0.91835422, 0.92845904, 0.91425664, 0.92828322]),
 'test_mae': array([0.70097389, 0.70918358, 0.71321412, 0.70439837, 0.71412476]),
 'fit_time': (2.0460805892944336,
  1.7925360202789307,
  2.0117416381835938,
  1.8581533432006836,
  4.328710556030273),
 'test_time': (0.13876628875732422,
  0.2133939266204834,
  0.12741708755493164,
  0.1291942596435547,
  0.6985030174255371)}

In [ ]:
data

In [ ]:
# Here we split the dataset into train, validation, and test sets in order to perform our model fitting and tuning
train_data, temp_data = train_test_split(data, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Here, we save the datasets in the csv formats.
train_data.to_csv('train.csv', index=False)
val_data.to_csv('validation.csv', index=False)
test_data.to_csv('test.csv',index=False)

In [ ]:
data

,userId,movieId,rating
0,1,2,3.5
1,5,2,3.0
2,13,2,3.0
3,29,2,3.0
4,34,2,3.0
...,...,...,...
2308632,15617,109008,3.0
2308633,15617,109455,3.0
2308634,15617,109858,2.0
2308635,15617,110412,3.5


##Collaborative filtering model

In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163005 sha256=dc7b869f033980685837d7e91c02d299f14a513b7d5e7b09abf94c37588a5a13
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import GridSearchCV

In [ ]:
# Load the train dataset using Surprise library
reader = Reader(rating_scale=(1, 5))
train_data_surprise = Dataset.load_from_df(train_data[['userId', 'movieId', 'rating']], reader)

required_columns=['userId', 'movieId', 'rating']
val_data1= val_data[required_columns]

In [ ]:
print(val_data1.head())
print(val_data1.info())

In [ ]:
val_data1[['userId', 'movieId', 'rating']]

,userId,movieId,rating
970332,8575,1961,3.5
167077,664,1387,5.0
1768733,1817,3274,4.0
1342887,15359,3505,4.0
1468802,11090,3418,5.0
...,...,...,...
1688085,12679,1635,5.0
2155648,2905,3494,5.0
681147,9658,802,3.0
1983138,12066,3013,4.0


In [ ]:
# Train a user-based collaborative filtering model
sim_options = {'name': 'cosine', 'user_based': True}
knn_model = KNNBasic(sim_options=sim_options)
knn_model.fit(train_data_surprise.build_full_trainset())

# Make predictions on the validation set
val_pred = knn_model.test(val_data1[['userId', 'movieId', 'rating']])

NameError: name 'KNNBasic' is not defined

##Evalution metrices

In [ ]:
from surprise import accuracy
rmse = accuracy.rmse(val_pred)

NameError: name 'val_pred' is not defined

##Hyperparameter tuning

In [ ]:
# Here we define parameter grid for hyperparameter tuning
param_grid = {'k': [20, 30, 40]}

# To perform grid search cross-validation, we use the code below
gs = GridSearchCV(KNNBasic, param_grid, measures=['rmse'], cv=3)
gs.fit(train_data_surprise)

# Here, we get the best RMSE score and parameters
best_rmse = gs.best_score['rmse']
best_params = gs.best_params['rmse']

##Testing

In [ ]:
# Make predictions on the test set
test_pred = knn_model.test(test_data[['userId', 'movieId', 'rating']])

# Evaluate the model on the test set
test_rmse = accuracy.rmse(test_pred)

In [ ]:
movie_path='/content/drive/MyDrive/RecSys/movie.csv'
rating_path='/content/drive/MyDrive/RecSys/rating.csv'